In [9]:
from concurrent.futures import ThreadPoolExecutor
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import math
URL = "https://www.amazon.com/Blackout-Absolutely-Insulated-Draperies-Anti-Rust/dp/B089CQ2R2Y/ref=cm_cr_arp_d_product_top?ie=UTF8"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3",
    "Host": "www.amazon.com",
    "Accept-Encoding": "gzip, deflate",
}

response = requests.get(URL, headers=HEADERS)

soup = BeautifulSoup(response.content, "html.parser")

see_all_reviews_link = soup.find("a", {"data-hook": "see-all-reviews-link-foot"})["href"]


URL_ALL_REVIEWS="https://www.amazon.com/"+see_all_reviews_link


response = requests.get(URL_ALL_REVIEWS, headers=HEADERS)

soup = BeautifulSoup(response.content, "html.parser")


# Find the element that contains the ratings with reviews and extract the text
ratings_with_reviews_elem = soup.find("div", {"id": "filter-info-section"})
ratings_with_reviews_text = ratings_with_reviews_elem.get_text().strip()

#get just the reviews amount
reviews_count = re.search(r'(\d+,*\d*)\s+with reviews', ratings_with_reviews_text).group(1)




data = []


def scrape_reviews(page_url):
    response = requests.get(page_url, headers=HEADERS)
    soup = BeautifulSoup(response.text, 'html.parser')
    reviews = soup.find_all('div', {'data-hook': 'review'})
    page_data = []
    for review in reviews:
        profile_name = review.find("span", {"class": "a-profile-name"}).get_text()
        title_elem = review.find("a", {"data-hook": "review-title"})
        title = title_elem.get_text().strip() if title_elem else ""
        body = review.find("span", {"data-hook": "review-body"}).get_text().strip()
        stars_elem = review.find("i", {"data-hook": "review-star-rating"})
        stars = float(stars_elem.span.get_text().split()[0]) if stars_elem and stars_elem.span else None
        page_data.append([profile_name, title, body, stars])
    return page_data

with ThreadPoolExecutor() as executor:
    page_num=math.ceil(float(reviews_count.replace(",",""))/10)
    pages = [f"{URL_ALL_REVIEWS}+&pageNumber={i+1}" for i in range(page_num)]
    results = executor.map(scrape_reviews, pages)
    for result in results:
        data += result

https://www.amazon.com//Blackout-Absolutely-Insulated-Draperies-Anti-Rust/product-reviews/B089CQ2R2Y/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews


In [10]:
df = pd.DataFrame(data, columns=['Name', 'Title', 'Body', 'Stars'])
df.head()

,Name,Title,Body,Stars
0,keyrse,"Quality, light blocking, color and style",Received these today and put them up directly ...,5.0
1,AM,"Excellent quality, very good blackout",Update: inside photos added.I will update with...,5.0
2,Lexi,"Good for the price, surprisingly light-blocking",I was looking for a set of blackout curtains f...,4.0
3,L J,Perfect for our needs,We got the extra wide size for our large bay w...,5.0
4,Jess,Does the job!,The pros: was looking for white and they are ...,4.0


In [11]:
len(df)

2107